In [ ]:
# making rfm rank based on each Recency, Frequency, Monetary values (from July to December)

In [1]:
import os
import time
import datetime
from datetime import date
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

In [8]:
def ranking(value):
    if value < quintile.iloc[0]:
        return 1
    elif (value >= quintile.iloc[0] and value < quintile.iloc[1]):
        return 2
    elif (value >= quintile.iloc[1] and value < quintile.iloc[2]):
        return 3
    elif (value >= quintile.iloc[2] and value < quintile.iloc[3]):
        return 4
    else:
        return 5  

In [9]:
order_org["created"] = pd.to_datetime(order_org.created.str.slice(start=0, stop=10), format="%Y-%m-%d")
order_org["date_diff"] = ((datetime.datetime(2018,12,31))-(order_org.created))
order_org["recency"] = [x.days for x in order_org["date_diff"]]
order_org["month"] = [x.month for x in order_org["created"]]

# Buyer

In [10]:
s = 7
i = 0
pbar = tqdm(total=5, initial=i)
rfm_df_buyer = []

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

In [11]:
while i<=5 :
    order = order_org[order_org["month"]==s+i]
    buyer_r = pd.DataFrame(order.groupby("buyer")["recency"].min())
    buyer_f = pd.DataFrame({"freq":order.buyer.value_counts()})
    buyer_m = pd.DataFrame(order.groupby("buyer")["amount"].sum())
    buyer_RFM = pd.concat([buyer_r, buyer_f, buyer_m], axis=1, sort=False)
    buyer_RFM["buyer"] = buyer_RFM.index
    buyer_RFM.index.name = None
    buyer_RFM = buyer_RFM.reset_index()
    buyer_RFM = buyer_RFM.drop("index", axis=1)
    buyer_RFM = buyer_RFM[["buyer", "recency", "freq", "amount"]]
    buyer_RFM["recency"] = buyer_RFM["recency"]*-1+183
    scaled = scaler.fit_transform(buyer_RFM)
    buyer_scaled = pd.DataFrame(scaled, columns = ["buyer", "recency", "freq", "amount"])
    buyer_name = buyer_RFM[["buyer"]]
    a = pd.concat([buyer_scaled[["recency", "freq", "amount"]], buyer_name], axis=1)
    buyer_scaled["rfm_value"]=(buyer_scaled["freq"]*1000)+((buyer_scaled["recency"]*-1+183)*100)+((buyer_scaled["amount"]*10)*10)
    quintile = (buyer_scaled["rfm_value"].quantile([.2, .4, .6, .8]))
    buyer_scaled["rfm_rank"] = ((buyer_scaled["rfm_value"])).apply(ranking)
    quintile = (buyer_scaled["recency"].quantile([.2, .4, .6, .8]))
    buyer_scaled["r_rank"] = ((buyer_scaled["recency"])).apply(ranking)
    quintile = (buyer_scaled["freq"].quantile([.2, .4, .6, .8]))
    buyer_scaled["f_rank"] = ((buyer_scaled["freq"])).apply(ranking)
    quintile = (buyer_scaled["amount"].quantile([.2, .4, .6, .8]))
    buyer_scaled["m_rank"] = ((buyer_scaled["amount"])).apply(ranking)
    buyer_final = pd.concat([buyer_RFM, buyer_scaled[["recency", "r_rank" ,"freq", "f_rank", "amount", "m_rank", "rfm_value", "rfm_rank"]]],axis=1)
    buyer_final["month"] = s+i
    rfm_df_buyer.append(buyer_final)
    pbar.update(1)
    i+=1
pbar.close()

C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
 20%|████████████████▊                                                                   | 1/5 [00:02<00:08,  2.14s/it]C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
 40%|█████████████████████████████████▌                                                  | 2/5 [00:02<00:04,  1.62s/it]C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:03<00:02,  1

In [12]:
rfm_df_buyer = pd.concat(rfm_df_buyer, axis=0)
rfm_df_buyer.columns = ["buyer", "recency", "freq", "amount", "recency_s", "r_rank", "freq_s", "f_rank", "amount_s", "m_rank", "rfm_value", "rfm_rank", "month"]
rfm_df_buyer.head()

,buyer,recency,freq,amount,recency_s,r_rank,freq_s,f_rank,amount_s,m_rank,rfm_value,rfm_rank,month
0,6400,21,5,135000.0,0.700000,3,0.006088,2,0.007866,3,18236.874904,2,7
1,7613,5,24,291000.0,0.166667,1,0.035008,4,0.017198,4,18320.060750,5,7
2,7616,14,7,84000.0,0.466667,2,0.009132,2,0.004815,2,18262.947299,3,7
3,8209,30,381,5543500.0,1.000000,5,0.578387,5,0.331399,5,18811.526523,5,7
4,8507,17,10,128000.0,0.566667,2,0.013699,3,0.007448,3,18257.776714,3,7


In [14]:
rfm_df_buyer.to_csv("buyerRFM_monthly_ind.csv")

# Product

In [19]:
order_org["created"] = pd.to_datetime(order_org.created.str.slice(start=0, stop=10), format="%Y-%m-%d")
order_org["date_diff"] = ((datetime.datetime(2018,12,31))-(order_org.created))
order_org["recency"] = [x.days for x in order_org["date_diff"]]
order_org["month"] = [x.month for x in order_org["created"]]

In [20]:
s = 7
i = 0
pbar = tqdm(total=5, initial=i)
rfm_df_product = []

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

In [22]:
while i<=5 :
    order = order_org[order_org["month"]==s+i]
    product_r = pd.DataFrame(order.groupby("product")["recency"].min())
    product_f = pd.DataFrame({"freq":order["product"].value_counts()})
    product_m = pd.DataFrame(order.groupby("product")["amount"].sum())
    product_RFM = pd.concat([product_r, product_f, product_m], axis=1, sort=False)
    product_RFM["product"] = product_RFM.index
    product_RFM.index.name = None
    product_RFM = product_RFM.reset_index()
    product_RFM = product_RFM.drop("index", axis=1)
    product_RFM = product_RFM[["product", "recency", "freq", "amount"]]
    product_RFM["recency"] = product_RFM["recency"]*-1+183
    scaled = scaler.fit_transform(product_RFM)
    product_scaled = pd.DataFrame(scaled, columns = ["product", "recency", "freq", "amount"])
    product_name = product_RFM[["product"]]
    a = pd.concat([product_scaled[["recency", "freq", "amount"]], product_name], axis=1)
    product_scaled["rfm_value"]=(product_scaled["freq"]*1000)+((product_scaled["recency"]*-1+183)*100)+((product_scaled["amount"]*10)*10)
    quintile = (product_scaled["rfm_value"].quantile([.2, .4, .6, .8]))
    product_scaled["rfm_rank"] = ((product_scaled["rfm_value"])).apply(ranking)
    quintile = (product_scaled["recency"].quantile([.2, .4, .6, .8]))
    product_scaled["r_rank"] = ((product_scaled["recency"])).apply(ranking)
    quintile = (product_scaled["freq"].quantile([.2, .4, .6, .8]))
    product_scaled["f_rank"] = ((product_scaled["freq"])).apply(ranking)
    quintile = (product_scaled["amount"].quantile([.2, .4, .6, .8]))
    product_scaled["m_rank"] = ((product_scaled["amount"])).apply(ranking)
    product_final = pd.concat([product_RFM, product_scaled[["recency", "r_rank" ,"freq", "f_rank", "amount", "m_rank", "rfm_value", "rfm_rank"]]],axis=1)
    product_final["month"] = s+i
    rfm_df_product.append(product_final)
    pbar.update(1)
    i+=1
pbar.close()

C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
 20%|████████████████▊                                                                   | 1/5 [00:17<01:11, 17.89s/it]C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
 40%|█████████████████████████████████▌                                                  | 2/5 [00:21<00:40, 13.46s/it]C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:24<00:20, 10

In [23]:
rfm_df_product = pd.concat(rfm_df_product, axis=0)
rfm_df_product.columns = ["product", "recency", "freq", "amount", "recency_s", "r_rank", "freq_s", "f_rank", "amount_s", "m_rank", "rfm_value", "rfm_rank", "month"]
rfm_df_product.head()

,product,recency,freq,amount,recency_s,r_rank,freq_s,f_rank,amount_s,m_rank,rfm_value,rfm_rank,month
0,102870,21,1,5000.0,0.700000,4,0.000000,3,0.002901,1,18230.290065,2,7
1,109661,25,3,24000.0,0.833333,5,0.024096,5,0.016679,3,18242.430927,2,7
2,109662,29,1,19000.0,0.966667,5,0.000000,3,0.013053,3,18204.638627,1,7
3,193533,7,3,72000.0,0.233333,2,0.024096,5,0.051487,5,18305.911711,5,7
4,213260,18,2,7000.0,0.600000,3,0.012048,4,0.004351,1,18252.483291,3,7


In [25]:
rfm_df_product.to_csv("productRFM_monthly_ind.csv")

# Brand

In [26]:
order_org["created"] = pd.to_datetime(order_org.created.str.slice(start=0, stop=10), format="%Y-%m-%d")
order_org["date_diff"] = ((datetime.datetime(2018,12,31))-(order_org.created))
order_org["recency"] = [x.days for x in order_org["date_diff"]]
order_org["month"] = [x.month for x in order_org["created"]]
order_org.head()

,ord,orddtl,buyer,brand,product,product_option,quantity,price,amount,created,date_diff,recency,month
0,162487,2641747,34990,6584,1047047,3646890,1,30000.0,30000.0,2018-07-01,183 days,183,7
1,162490,2641766,54804,9400,940522,3247569,1,6000.0,6000.0,2018-07-01,183 days,183,7
2,162490,2641765,54804,7966,1029752,3588948,1,34000.0,34000.0,2018-07-01,183 days,183,7
3,162490,2641751,54804,192,1045597,3642718,1,22000.0,22000.0,2018-07-01,183 days,183,7
4,162490,2641767,54804,11506,1066000,3716015,1,21000.0,21000.0,2018-07-01,183 days,183,7


In [29]:
s = 7
i = 0
pbar = tqdm(total=5, initial=i)
rfm_df_brand = []


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

In [30]:
while i<=5 :
    order = order_org[order_org["month"]==s+i]
    brand_r = pd.DataFrame(order.groupby("brand")["recency"].min())
    brand_f = pd.DataFrame({"freq":order["brand"].value_counts()})
    brand_m = pd.DataFrame(order.groupby("brand")["amount"].sum())
    brand_RFM = pd.concat([brand_r, brand_f, brand_m], axis=1, sort=False)
    brand_RFM["brand"] = brand_RFM.index
    brand_RFM.index.name = None
    brand_RFM = brand_RFM.reset_index()
    brand_RFM = brand_RFM.drop("index", axis=1)
    brand_RFM = brand_RFM[["brand", "recency", "freq", "amount"]]
    brand_RFM["recency"] = brand_RFM["recency"]*-1+183
    scaled = scaler.fit_transform(brand_RFM)
    brand_scaled = pd.DataFrame(scaled, columns = ["brand", "recency", "freq", "amount"])
    brand_name = brand_RFM[["brand"]]
    a = pd.concat([brand_scaled[["recency", "freq", "amount"]], brand_name], axis=1)
    brand_scaled["rfm_value"]=(brand_scaled["freq"]*1000)+((brand_scaled["recency"]*-1+183)*100)+((brand_scaled["amount"]*10)*10)
    quintile = (brand_scaled["rfm_value"].quantile([.2, .4, .6, .8]))
    brand_scaled["rfm_rank"] = ((brand_scaled["rfm_value"])).apply(ranking)
    quintile = (brand_scaled["recency"].quantile([.2, .4, .6, .8]))
    brand_scaled["r_rank"] = ((brand_scaled["recency"])).apply(ranking)
    quintile = (brand_scaled["freq"].quantile([.2, .4, .6, .8]))
    brand_scaled["f_rank"] = ((brand_scaled["freq"])).apply(ranking)
    quintile = (brand_scaled["amount"].quantile([.2, .4, .6, .8]))
    brand_scaled["m_rank"] = ((brand_scaled["amount"])).apply(ranking)
    brand_final = pd.concat([brand_RFM, brand_scaled[["recency", "r_rank" ,"freq", "f_rank", "amount", "m_rank", "rfm_value", "rfm_rank"]]],axis=1)
    brand_final["month"] = s+i
    rfm_df_brand.append(brand_final)
    pbar.update(1)
    i+=1
pbar.close()

C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)

 20%|████████████████▊                                                                   | 1/5 [00:01<00:06,  1.72s/it]C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)

 40%|█████████████████████████████████▌                                                  | 2/5 [00:02<00:04,  1.34s/it]C:\Users\irie9\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:02<00:02,

In [31]:
rfm_df_brand = pd.concat(rfm_df_brand, axis=0)
rfm_df_brand.columns = ["brand", "recency", "freq", "amount", "recency_s", "r_rank", "freq_s", "f_rank", "amount_s", "m_rank", "rfm_value", "rfm_rank", "month"]
rfm_df_brand.head()

,brand,recency,freq,amount,recency_s,r_rank,freq_s,f_rank,amount_s,m_rank,rfm_value,rfm_rank,month
0,3,13,3,15000.0,0.433333,1,0.002963,2,0.002011,1,18259.830775,4,7
1,21,24,3,118000.0,0.800000,3,0.002963,2,0.017948,3,18224.757795,2,7
2,23,11,4,60000.0,0.366667,1,0.004444,2,0.008974,2,18268.675194,4,7
3,38,30,11,192000.0,1.000000,5,0.014815,4,0.029398,4,18217.754626,1,7
4,48,18,4,253000.0,0.600000,2,0.004444,2,0.038836,4,18248.328090,3,7


In [32]:
rfm_df_brand.to_csv("brandRFM_monthly_ind.csv")